In [1]:
import os
import numpy as np
import tensorflow as tf
from .kvae import KalmanVariationalAutoencoder
from .kvae.utils import reload_config, get_train_config
from .data import load_openl3_time_series

from .data_aggr import medoid

In [2]:
import sys
sys.argv = ['foo.py']

In [3]:
config = get_train_config()                                
# To reload a saved model                                  
config = reload_config(config.FLAGS)                       


In [4]:
full_ts_path = '/scratch/jtc440/sonyc-kalman/full_data.npy'
hdf5_path = '/beegfs/work/sonyc/features/openl3/2017/sonycnode-b827eb2a1bce.sonyc_features_openl3.h5'

In [5]:
config.log_dir = '/scratch/jtc440/sonyc-kalman'

In [6]:
if os.path.exists(full_ts_path):
    X = np.load(full_ts_path)
else:
    X, mask = load_openl3_time_series(hdf5_path, aggr_func=medoid, delta_mins=60)
    np.save(full_ts_path, X)

In [7]:
import librosa

X_train_full = np.ascontiguousarray(X[X.shape[0]//4:])
X_test_full = np.ascontiguousarray(X[:X.shape[0]//4])

n_timesteps = 24
hop_length = n_timesteps // 4
train_data = librosa.util.frame(X_train_full, frame_length=n_timesteps, hop_length=hop_length, axis=0)
test_data = librosa.util.frame(X_test_full, frame_length=n_timesteps, hop_length=hop_length, axis=0)



In [8]:
os.environ['CUDA_VISIBLE_DEVICES'] = config.gpu

In [9]:
with tf.Session() as sess:                                                   
    model = KalmanVariationalAutoencoder(train_data, test_data, config, sess)
                                                                             
    model.build_model().build_loss().initialize_variables()                  
    err = model.train()                                                      


W1118 19:16:13.079126 47468108574336 deprecation_wrapper.py:119] From /home/jtc440/dev/sonyc-kalman/sonyc_kalman/kvae/KalmanVariationalAutoencoder.py:65: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1118 19:16:13.084347 47468108574336 deprecation_wrapper.py:119] From /home/jtc440/dev/sonyc-kalman/sonyc_kalman/kvae/KalmanVariationalAutoencoder.py:93: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1118 19:16:13.457742 47468108574336 deprecation_wrapper.py:119] From /home/jtc440/dev/sonyc-kalman/sonyc_kalman/kvae/utils/nn.py:13: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W1118 19:16:13.467852 47468108574336 deprecation.py:323] From /home/jtc440/dev/sonyc-kalman/sonyc_kalman/kvae/KalmanVariationalAutoencoder.py:196: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This cla

Epoch 0, ELBO 1776.57, log_probs [-32.66, -8.80, -102.19, 3.44], elbo_vae 6390.92, took 10.57s
Hamming distance. x_imputed: 0.01031, x_filtered: 0.01034, x_gen_det: 0.01021, baseline: 0.01172. 
Normalized RMSE. a_imputed: 0.045, a_gen_det: 0.085
-- TEST, ELBO 7839.12, log_probs [-7.54, -0.03, -4.45, 1.96], elbo_vae 7849.18, took 0.14s
Epoch 1, ELBO 2329.05, log_probs [-5.22, 0.59, -1.12, 1.47], elbo_vae 7779.28, took 1.90s
Epoch 2, ELBO 2331.16, log_probs [-3.63, 0.73, -0.63, 1.05], elbo_vae 7780.35, took 1.91s
Epoch 3, ELBO 2331.65, log_probs [-3.02, 0.76, -0.58, 0.84], elbo_vae 7780.40, took 1.90s
Epoch 4, ELBO 2331.81, log_probs [-2.61, 0.77, -0.61, 0.69], elbo_vae 7780.12, took 1.95s
Epoch 5, ELBO 2331.92, log_probs [-2.31, 0.78, -0.60, 0.57], elbo_vae 7779.79, took 1.91s
Epoch 6, ELBO 2331.85, log_probs [-2.11, 0.78, -0.65, 0.47], elbo_vae 7779.42, took 1.95s
Epoch 7, ELBO 2332.21, log_probs [-1.98, 0.78, -0.53, 0.43], elbo_vae 7779.91, took 1.91s
Epoch 8, ELBO 2332.31, log_probs 

Hamming distance. x_imputed: 0.00945, x_filtered: 0.00928, x_gen_det: 0.00927, baseline: 0.01172. 
Normalized RMSE. a_imputed: 0.769, a_gen_det: 0.744
-- TEST, ELBO 7850.65, log_probs [-0.73, 0.82, -0.48, 0.70], elbo_vae 7850.34, took 0.16s
Negative lower_bound on the test set: -7850.649
